The intent of this notebook is to document different types of machine learning approaches and algorithms.

# Supervised learning
Wikipedia has an excellent definition:

**Supervised learning** is the machine learning task of inferring a function from labeled training data.
The training data consist of a set of training examples. In supervised learning, each example is a pair consisting of an input object (typically a vector) and a desired output value (also called the supervisory signal). A supervised learning algorithm analyzes the training data and produces an inferred function, which can be used for mapping new examples. An optimal scenario will allow for the algorithm to correctly determine the class labels for unseen instances. This requires the learning algorithm to generalize from the training data to unseen situations in a "reasonable" way 

## Classification ##
### Support Vector Machines (SVMs) ###
SVMs are a set of learning algorithms for to do classification. In a nutshell, SVMs try to find a hyperplane that maximes the distance between clusters of data.

Visually:

![SVM Example](svm/svm-example.png)

The points/vectors that define where the hyperplane fits are called the **support vectors** (they're "supporting" the maximum distance between the clusters and the plane).


![Support Vectors](svm/support-vectors.png)


Computing SVMs comes down to solving a differential equation: finding a hyperplane that minimizes the distance between the hyperplane and all potential support vectors in their respective clusters. There are multiple algorithms for solving this.

![Support Vectors](svm/svm-minimize.png)

Note that a tricky part of defining an SVMs is often finding the right feature space. What might be hard in one dimension, might become easy in a higher dimension that represents the right features:

![Support Vectors](svm/input-feature-space.png)




### Neural Networks ###

Neural network example, with input, output and hidden layers:

![Neutral Network](nn/neural_net.png)

Individual **neurons** are just sum or **"transfer"** functions:

![Sum Function](nn/sum-function.png)

In some cases they have activation **"sigmoid"** functions that will output constant or variable values if the output of the neuron reaches a certain threshold:

![Activation Function](nn/activation-function.png)



#### Types ####
There are different neural network types:

![Neural Network Types](nn/neural-network-types.jpg)
    
    
Perceptron: no hidden layers, only input and output.
Feed Forward: No cycles or loops in the network.

Recurrent Neural Network (RNN): also propagate data from later processing stages to earlier stages.

    